In [25]:
# Goal is to create a single dataset to pass to the lstm
import os
import json
import music21 as m21

In [26]:
env = m21.environment.Environment()
env['musescoreDirectPNGPath'] = r'C:/Program Files/MuseScore 4/bin/MuseScore4.exe'
env['musicxmlPath'] = r'C:/Program Files/MuseScore 4/bin/MuseScore4.exe'

# Create constants
KERN_DATASET_PATH = "deutschl/test"
SAVE_DIR = "dataset"
SINGLE_FILE_DATASET = "file_dataset"
MAPPING_PATH = "mapping.json"
ACCEPTABLE_DURATIONS = [0.25, 
                        0.5, 
                        0.75,
                        1.0,
                        1.5,
                        2,
                        3,
                        4
                        ]
SEQUENCE_LENGTH = 64

# Load all songs from the data set
def load_songs_in_kern(dataset_path):
    songs = []
    for path, subdirs, files in os.walk(dataset_path):
        for file in files:
            if file[-3:] == "krn":
                song=m21.converter.parse(os.path.join(path, file))
                songs.append(song)
    return songs


# To check if the duration is in the acceptable duration
def has_acceptable_durations(song, acceptable_durations):
    for note in song.flat.notesAndRests:   #flatten data into single list
        if note.quarterLength not in acceptable_durations:
            return False
    return True

# Transpose songs to C maj and A minor because we don't need to know all different keys - just reduce everything to C major and A minor so the model doesn't have to generalize to more keys (Uses less data)
def transpose(song):

    # get key from the song correctly
    parts = song.getElementsByClass(m21.stream.Part)
    measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measures_part0[0][4] 

    # estimate key using music21
    if not isinstance(key, m21.key.Key):
        key = song.analyze("key")

    print(key)

    # get interval for transposition. E.g., Bmaj -> Cmaj
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))

    # transpose song by calculated interval
    transposed_song = song.transpose(interval)

    return transposed_song

def encode_song(song, time_step = 0.25):
    # pitch = 60, d=1.0 -? [60, "_", "_", "_"]

    encoded_song = [] # stores all notes and rests in the time series

    for event in song.flat.notesAndRests:
        # handle notes
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi # is 60
        # handle rests
        elif isinstance(event, m21.note.Rest):
            symbol = "r"

        # convert the note/rest into time series notation
        steps = int(event.duration.quarterLength/time_step)
        for step in range(steps):
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")

    # cast encoded song to a str
    encoded_song = " ".join(map(str, encoded_song)) # join all items in encoded song list

    return encoded_song

def preprocess(dataset_path):
    print("Loading songs...")
    songs = load_songs_in_kern(dataset_path)
    print(f"Loaded {len(songs)} songs.")

    for i, song in enumerate(songs):
        # Filter out songs that have no-acceptable durations
        if not has_acceptable_durations(song, ACCEPTABLE_DURATIONS):
            continue

        # transpose songs to c major or a minor
        song = transpose(song)

        # encoded songs with music time series representation
        encoded_song = encode_song(song)

        # save songs to text file
        save_path = os.path.join(SAVE_DIR, str(i))
        with open(save_path, "w") as fp:
            fp.write(encoded_song)

# Open song file
def load(file_path):
    with open(file_path, "r") as fp:
        song = fp.read()
    return song

# Create a string to add all the different songs from the dataset folder
def create_single_file_dataset(dataset_path, file_dataset_path, sequence_length):
    new_song_delimiter = "/ " * sequence_length
    songs = ""

    # load encoded songs and add delimiters
    for path, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            song = load(file_path)
            songs = songs + song + " " + new_song_delimiter
    
    songs = songs[:-1]

    # save string that contains all dataset
    with open(file_dataset_path, "w") as fp:
        fp.write(songs)

    return songs

# Identigy the vocabulary or all symbols identified in the dataset
def create_mapping(songs, mapping_path):
    mappings = {}

    # Identify the vocabulary
    songs = songs.split()
    vocabulary = list(set(songs))

    # create mappings
    for i, symbol in enumerate(vocabulary):
        mappings[symbol] = i

    # save vocabulary to a json file
    with open(mapping_path, "w") as fp:
        json.dump(mappings, fp)

def main():
    preprocess(KERN_DATASET_PATH)
    songs = create_single_file_dataset(SAVE_DIR, SINGLE_FILE_DATASET, SEQUENCE_LENGTH)
    create_mapping(songs, MAPPING_PATH)

if __name__ == "__main__":
    main()
    '''
    # load songs
    songs = load_songs_in_kern(KERN_DATASET_PATH)
    print(f"Loaded {len(songs)} songs.")
    song = songs[0]
    filepath = 'output'

    preprocess(KERN_DATASET_PATH)

    # transpose song
    transposed_song = transpose(song)
    transposed_song.write('musicxml', filepath)
    song.show()
    transposed_song.show()
    os.startfile(filepath+".musicxml")
    '''

Loading songs...
Loaded 12 songs.
e minor
e minor
F major
C major
g minor
b minor
e minor
F major
C major
e minor
F major
C major


c:\Users\Noah\AppData\Local\Programs\Python\Python310\lib\site-packages\music21\stream\base.py:3675: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
